In [1]:
#pip install -r requirements.txt

In [27]:
import yaml
import json
import os
from alfred_git_data import PRDataset, PR
from tqdm import tqdm
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)
#logging.basicConfig(level=logging.INFO)
from IPython.display import display, HTML


In [3]:
#pip install pandas

In [4]:
import json
import gzip
def load_json_from_gzip(gzipped_filepath):
    """
    Loads a JSON file gzipped.

    Args:
        zip_filepath (str): The path to the gzipped file.

    Returns:
        dict: The JSON data as a Python dictionary, or None if an error occurs.
    """
    try:
        with gzip.open(gzipped_filepath, 'r') as z:
                data = json.load(z)
                return data
    except FileNotFoundError:
        print(f"Error: GZIP file not found: {gzipped_filepath}")
        return None
    except Exception as e:
         print(f"An unexpected error occurred: {e}")
         return None

In [ ]:
json_files = ["dataset/{dataset_to_load}"]
data_objs = {}
for json_file in json_files:
    if json_file.endswith(".gz"):
        data = load_json_from_gzip(json_file)
    else:
        with open(json_file, "r") as f:
            data = json.load(f)
    if data is not None:
        print(f"Loaded {len(data)} records from {json_file}")
        dst_pr = PRDataset(**data)
        data_objs[json_file] = dst_pr
    else:
        print(f"Failed to load data from {json_file}")


In [61]:
import pandas as pd
from alfred_git_data import CommentType
def split_comments_by_type(comments):
        issue_comments = [comment for comment in comments if comment.type == CommentType.issue]
        review_comments = [comment for comment in comments if comment.type == CommentType.review]
        file_comments = [comment for comment in comments if comment.type == CommentType.filec]
        return issue_comments, review_comments, file_comments

def collate_use_case_stats(PR: PR):
    """
    Collates use case statistics from multiple PRDataset objects.

    Args:
        PR (PR): A PR object.

    Returns:
        Dict: A Dict containing the aggregated statistics.
    """
    result = {'cross_reference': 0.0,
              'network_configuration': 0.0,
              'iam_policy': 0.0,
              'modification_to_module': 0.0,
              'reference_to_variable': 0.0,
              'module_usage': 0.0,
              'terraform_best_practice': 0.0}
    if PR.use_cases is None or not hasattr(PR, 'use_cases'):
        # Handle case where use_cases is None
        return result
    dfu = pd.DataFrame(PR.use_cases)
    num_files = len(PR.files)
    col_keep = [col for col in dfu.columns.tolist() if not col.endswith("file")]
    dfs = dfu[col_keep].sum()/num_files
    #dfs["#files"] = num_files
    result.update(dfs.to_dict())  # Update the result dictionary with the calculated values
    return result

def get_stats_df(dst_pr):
    # Initialize lists to store metrics
    num_files = []
    num_comments = []
    num_issue_comments = []
    num_review_comments = []
    num_file_comments = []
    num_commits = []
    num_pr = []
    use_case_stats_list = {}

    # Loop over dst_pr to extract metrics
    for pr in tqdm(dst_pr.PRs):
        num_pr.append(pr.pr_number)
        num_files.append(len(pr.files))
        num_comments.append(len(pr.comments))
        issue_comments, review_comments, file_comments = split_comments_by_type(comments=pr.comments)
        num_issue_comments.append(len(issue_comments))
        num_review_comments.append(len(review_comments))
        num_file_comments.append(len(file_comments))
        num_commits.append(len(pr.commits))
        use_case_stats = collate_use_case_stats(pr)
        for k, v in use_case_stats.items():
            # Aggregate use case stats
            if k not in use_case_stats_list:
                use_case_stats_list[k] = []
            use_case_stats_list[k].append(v)



    # Create a DataFrame with the metrics
    base_info = {
         'PR' : num_pr,
        '#tf_files': num_files,
        '#comments': num_comments,
        '#issue_comments': num_issue_comments,
        '#review_comments': num_review_comments,
        '#file_comments': num_file_comments,
        '#commits': num_commits
    }
    for k, v in use_case_stats_list.items():
        # Add each use case stat to the base_info
        base_info[k] = v
    df = pd.DataFrame(base_info)
    return df
#df.head()  # Display the first few rows of the DataFrame

In [62]:
#df = get_stats_df(data_objs["dataset/azure_terraform.json"])
#display(HTML(df.head().to_html()))  # Convert the DataFrame to an HTML table

In [63]:
def get_comments_as_df(dst_pr: PRDataset):
    # Initialize lists to store metrics
    pr_numbers = []
    comment_types = []
    comment_bodies = []

    # Loop over dst_pr to extract metrics
    for pr in tqdm(dst_pr.PRs):
        for comment in pr.comments:
            pr_numbers.append(pr.pr_number)
            comment_types.append(comment.type)
            comment_bodies.append(comment.comment)

    # Create a DataFrame with the metrics
    df = pd.DataFrame({
        'PR' : pr_numbers,
        'type': comment_types,
        'body': comment_bodies
    })
    return df

In [64]:
obj_name = {}
df = get_stats_df(data_objs[obj_name])
display(HTML(df.head().to_html()))  # Convert the DataFrame to an HTML table
df.to_excel("summary_pr_cisco_eti_path_terraform_infra_repo.xlsx", index=False)
#df_comments = get_comments_as_df(data_objs[obj_name])
#display(HTML(df_comments.head(1).to_html()))  # Convert the DataFrame to an HTML table

100%|██████████| 713/713 [00:00<00:00, 3324.63it/s]


,PR,#tf_files,#comments,#issue_comments,#review_comments,#file_comments,#commits,cross_reference,network_configuration,iam_policy,modification_to_module,reference_to_variable,module_usage,terraform_best_practice
0,1,64,110,109,1,0,42,0.890625,0.265625,0.3125,0.078125,0.625000,0.125000,0.828125
1,2,10,25,25,0,0,9,1.000000,0.400000,0.6000,0.000000,0.700000,0.000000,0.700000
2,3,5,23,14,5,4,10,1.000000,0.600000,0.2000,0.200000,0.800000,0.000000,0.800000
3,4,2,7,7,0,0,1,1.000000,0.500000,0.0000,0.500000,0.500000,0.500000,0.500000
4,5,15,20,20,0,0,9,1.000000,0.333333,0.2000,0.466667,0.733333,0.466667,0.866667


In [41]:
# Load the JSON file as a dictionary
json_file = "dataset/rating_new_both(2).json"
with open(json_file, 'r') as z:
    json_data = json.load(z)
# Check if the data was loaded successfully
if json_data is not None:
    print(f"Successfully loaded JSON data from {json_file}")
else:
    print(f"Failed to load JSON data from {json_file}")

Successfully loaded JSON data from dataset/rating_new_both(2).json


In [70]:
def get_use_cases_from_PRDataset(dst_or: PRDataset):
    """
    Extracts use cases from a PRDataset object.

    Args:
        dst_or (PRDataset): The PRDataset object to extract use cases from.

    Returns:
        Dict: A dictionary where keys are PR numbers and values are the corresponding use cases.
    """
    use_cases = {}
    for pr in dst_or.PRs:
        if pr.use_cases is not None:
            use_cases[pr.pr_number] = pr.use_cases
    # Flatten the use cases into a list of dictionaries
    # This will help in aggregating and analyzing the use cases
    # Note: This assumes that each PR's use_cases is a list of dictionaries
    # If use_cases is a dictionary, you might want to flatten it differently
    return use_cases
prs_to_use_cases = get_use_cases_from_PRDataset(data_objs[obj_name])

In [78]:
prs_to_use_cases.keys()

dict_keys([1, 2, 3, 4, 5, 6, 8, 12, 9, 14, 15, 17, 16, 11, 10, 13, 19, 21, 20, 23, 22, 24, 26, 30, 32, 28, 29, 27, 31, 35, 36, 37, 38, 39, 43, 41, 47, 45, 44, 48, 51, 52, 53, 56, 59, 60, 61, 62, 63, 65, 66, 68, 70, 71, 46, 72, 74, 76, 73, 75, 77, 78, 79, 80, 55, 81, 83, 84, 85, 86, 40, 88, 90, 91, 92, 95, 96, 97, 98, 99, 100, 101, 102, 94, 105, 106, 103, 108, 114, 115, 116, 117, 113, 118, 119, 120, 87, 122, 121, 124, 112, 123, 127, 128, 130, 129, 131, 133, 132, 134, 126, 135, 136, 137, 140, 144, 146, 147, 148, 149, 151, 150, 152, 153, 141, 156, 160, 161, 162, 163, 157, 167, 166, 169, 171, 172, 175, 165, 176, 174, 178, 179, 170, 182, 184, 183, 187, 188, 185, 189, 190, 191, 193, 195, 194, 196, 197, 198, 200, 192, 201, 203, 202, 204, 205, 206, 207, 209, 210, 211, 212, 214, 154, 218, 216, 219, 220, 224, 225, 226, 228, 230, 231, 232, 233, 234, 223, 235, 237, 238, 239, 236, 240, 241, 245, 229, 242, 246, 249, 248, 244, 250, 227, 252, 253, 251, 254, 222, 215, 258, 255, 262, 263, 265, 266, 267,

In [ ]:
from copy import deepcopy
df2 = df.set_index("PR")
rest_list = []
counter = 0

base_cases = {'cross_reference': None,
               'network_configuration': None,
               'iam_policy': None,
               'modification_to_module': None,
               'reference_to_variable': None,
               'module_usage': None,
               'terraform_best_practice': None}
for key,val in json_data.items():
    pr_num = int(key.split("pr_number_")[-1])
    if pr_num in prs_to_use_cases:
        print("Found PR number:", pr_num)
        # The PR number exists in the use cases
        # You can now process this entry further if needed
        local_use_case = prs_to_use_cases[pr_num]
    else:
        print("Skipping PR number:", pr_num)
        continue
    for key2 , val2 in val.items():
        if ".tf" in key2:
            for rating_obj in val2:
                if rating_obj is not None and rating_obj.get("comment") is not None:
                    # Extract the body of the comment
                    body = rating_obj["comment"]
                    rating = rating_obj.get("rating", None)
                    for cmt in local_use_case:
                        if cmt["local_file"] == key2:
                            rest = deepcopy(base_cases)
                            print(key2, "matched with local file in use case for PR", pr_num)
                            for c in rest.keys():
                                try:
                                    if cmt[c] > 0:
                                        rest[c] = rating
                                except Exception as e:
                                    print(f"ERROR {c} {cmt}")
                            rest["PR"] = pr_num
                            rest["local_file"] = key2
                            rest_list.append(rest)  # Append the updated dictionary to the list                              
                else:
                    print(rating_obj)
            # Handle file-specific keys
            #print(key2)
            #print(val2)

Found PR number: 190
Found PR number: 189
Found PR number: 188
Found PR number: 187
Skipping PR number: 186
Found PR number: 185
Found PR number: 184
Found PR number: 183
Found PR number: 182
Skipping PR number: 181
Skipping PR number: 180
Found PR number: 179
Found PR number: 178
Skipping PR number: 177
Found PR number: 176
Found PR number: 175
Found PR number: 174
Skipping PR number: 173
Found PR number: 172
Found PR number: 171
Found PR number: 170
Found PR number: 169
None
Skipping PR number: 168
Found PR number: 167
Found PR number: 166
Found PR number: 165
Skipping PR number: 164
Found PR number: 163
Found PR number: 162
Found PR number: 161
Found PR number: 160
Skipping PR number: 159
Skipping PR number: 158
Found PR number: 157
Found PR number: 156
Skipping PR number: 155
Found PR number: 154
Found PR number: 153
Found PR number: 152
Found PR number: 151
Found PR number: 150
Found PR number: 149
Found PR number: 148
Found PR number: 147
Found PR number: 146
Skipping PR number: 

In [93]:
df_rating_by_case = pd.DataFrame(rest_list)

In [96]:
df_rating_by_case['local_file'].nunique()

21

In [97]:
df_rating_by_case.to_excel("review_cisco_eti_path_terraform_infra_repo_with_ratings.xlsx", index=False)

In [81]:
prs_to_use_cases[8]

[{'cross_reference': 1,
  'network_configuration': 1,
  'iam_policy': 1,
  'modification_to_module': 0,
  'reference_to_variable': 0,
  'module_usage': 0,
  'terraform_best_practice': 1,
  'local_file': 'aws_eticloud-preproduction_us-east-2_eks_eks-dev-3_main.tf',
  'original_file': 'aws/eticloud-preproduction/us-east-2/eks/eks-dev-3/main.tf'},
 {'cross_reference': 1,
  'network_configuration': 0,
  'iam_policy': 0,
  'modification_to_module': 0,
  'reference_to_variable': 0,
  'module_usage': 0,
  'terraform_best_practice': 0,
  'local_file': 'aws_eticloud-preproduction_us-east-2_eks_eks-dev-3_provider.tf',
  'original_file': 'aws/eticloud-preproduction/us-east-2/eks/eks-dev-3/provider.tf'}]

In [51]:
#df2.loc[int(pr_num), :]
val.keys()

dict_keys(['2744184134', '2744190602', '2744190648', 'aws_eticloud-scratch-c_eks_eks-butterscotch-1_main.tf', 'modules_eks_bastion.tf', 'modules_eks_keypair.tf', 'modules_eks_variables.tf', 'modules_eks_vpc-prefixlist.tf', 'modules_eks_vpc.tf', '2706949773', '2744191079', '2744197911', '2706957431', '2744198421', '2744204988', '2706968663', '2744205722', '2744212409', '2744212463', '2744213183', '2744218050', '2706983464', '2744218578', '2744224689', '2744224740', '2744225158', '2744233171', '2707010752', '2744233728', '2744241315', 'aws_eticloud-scratch-c_us-east-2_eks_eks-butterscotch-1_provider.tf', '2707019923', '2744241632', '2744246388', '2744246425', '2744246827', '2744252204', '2744252278', '2744253001', '2749036752', '2749037618', '2749045377', 'aws_eticloud-scratch-c_us-east-2_eks_eks-butterscotch-1_backend.tf', 'aws_eticloud-scratch-c_us-east-2_eks_eks-butterscotch-1_locals.tf', 'modules_eks_vpc-bastion-keypair.tf', 'modules_eks_vpc-bastion.tf', '2711336293', '2749053400', '

In [21]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = make_subplots(
    rows=2, cols=1,
    shared_xaxes=True,
    vertical_spacing=0.03,
    specs=[[{"type": "table"}],
           [{"type": "table"}],
           #[{"type": "scatter"}]
           ]
)
fig.add_trace(
    go.Table(
        header=dict(
            values=['PR', '#tf_files', '#comments', '#issue_comments', '#review_comments',
       '#file_comments', '#commits'],
            font=dict(size=10),
            align="left"
        ),
        cells=dict(
            values=[df[k].tolist() for k in df.columns],
            align = "left")
    ),
    row=1, col=1
)
fig.add_trace(
    go.Table(
        header=dict(
            values=['PR', 'type', 'body'],
            font=dict(size=10),
            align="left"
        ),
        cells=dict(
            values=[df_comments[k].tolist() for k in df.columns],
            align = "left")
    ),
    row=2, col=1
)

In [22]:
print(df_comments.columns)
print(df.columns)

In [14]:
#pip install --upgrade nbformat

In [25]:
dfu = pd.DataFrame(dst_pr.PRs[0].use_cases)
num_files = len(dst_pr.PRs[0].files)
col_keep = [col for col in dfu.columns.tolist() if not col.endswith("file")]
dfs = dfu[col_keep].sum()/num_files
dfs["#files"] = num_files


In [28]:
dfs.to_dict()

{'cross_reference': 0.890625,
 'network_configuration': 0.265625,
 'iam_policy': 0.3125,
 'modification_to_module': 0.078125,
 'reference_to_variable': 0.625,
 'module_usage': 0.125,
 'terraform_best_practice': 0.828125,
 '#files': 64.0}